In [47]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import os

In [48]:
trainpath = os.path.join("data", "trail_conll_spanglish.csv")

In [49]:
train = pd.read_csv(filepath, sep='\\t', names=["ID","SENTENCE","LABEL"])

train.head()

/Users/utkarshdwivedi/Documents/dl_projects/tf/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,ID,SENTENCE,LABEL
0,1,So that means tomorrow cruda segura lol,positive
1,2,Tonight peda segura,neutral
2,3,Eres tan mala vieja bruja interesada#jamming,negative
3,4,Yo kiero Pretzels lol,neutral
4,5,Fuck that ni ke el me vaya a mantener toda la ...,negative


In [50]:
test = pd.read_csv("sample_test.csv", names=["ID", "SENTENCE", "LABEL"])
test.head()

,ID,SENTENCE,LABEL
0,1,great books read very well,positive
1,2,boo hoo mar gaya bechara,negative


In [51]:
train.columns


Index(['ID', 'SENTENCE', 'LABEL'], dtype='object')

In [52]:
test.columns

Index(['ID', 'SENTENCE', 'LABEL'], dtype='object')

In [53]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    dtype=np.float32,
    max_features=5000
)


In [54]:
char_vectorizer.fit(train['SENTENCE'])
train_char_features = char_vectorizer.transform(train['SENTENCE'])

In [55]:
char_vectorizer.fit(test['SENTENCE'])
test_char_features = char_vectorizer.transform(test['SENTENCE'])

In [59]:
train_char_features.shape

(1999, 5000)

In [63]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
model_NB = MultinomialNB()

X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(train_char_features, train.LABEL, train_size=0.75)
model_NB.fit(X_train_tfidf, y_train_tfidf)
predictions_tfidf = model_NB.predict(X_test_tfidf)
accuracy_tfidf = accuracy_score(y_test_tfidf, predictions_tfidf)
accuracy_tfidf

0.492

In [72]:
from sklearn import svm
clf = svm.SVC(gamma='scale')
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(train_char_features, train.LABEL, train_size=0.75)
clf.fit(X_train_tfidf, y_train_tfidf)
predictions_tfidf = clf.predict(X_test_tfidf)
accuracy_tfidf = accuracy_score(y_test_tfidf, predictions_tfidf)
accuracy_tfidf

0.532

In [73]:
clf = svm.SVC(kernel="linear", gamma="auto")
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(train_char_features, train.LABEL, train_size=0.75)
clf.fit(X_train_tfidf, y_train_tfidf)
predictions_tfidf = clf.predict(X_test_tfidf)
accuracy_tfidf = accuracy_score(y_test_tfidf, predictions_tfidf)
accuracy_tfidf


0.542